In [1]:
%matplotlib inline

## Definicion del modelo LTC en PyTorch

In [ ]:
import torch #Libreria principal que carga todos los modulos de PyTorch
import torch.nn as nn #Liberia NN que permite la creacion de modelos de redes neuronales
import torch.nn.functional as F #Libreria F que permite cargar las diferentes funciones para las redes

class LTC(nn.Module):

    def __init__(self,nro_canales):
        super(LTC, self).__init__()
        #Capas convolucionales
        self.convolucion3d_1 = nn.Conv3d(in_channels=nro_canales,out_channels=64,kernel_size=3,stride=1,padding=1,bias=False)
        self.convolucion3d_2 = nn.Conv3d(64,128,kernel_size=3,stride=1,padding=1,bias=False)
        self.convolucion3d_3 = nn.Conv3d(128,256,kernel_size=3,stride=1,padding=1,bias=False)
        self.convolucion3d_4 = nn.Conv3d(256,256,kernel_size=3,stride=1,padding=1,bias=False)
        self.convolucion3d_5 = nn.Conv3d(256,256,kernel_size=3,stride=1,padding=1,bias=False)
        self.full_connected_1 = nn.Linear(256*t/16*w/32*h/32,2048) #Como defino la entrada de la primera fc
        self.full_connected_2 = nn.Linear(2048,2048)
        self.full_connected_3 = nn.Linear(2048,nro_clases)

    def forward(self, x):
        #Primera convolucion
        x = self.convolucion3d_1(x)
        x = F.relu(x)
        x = F.max_pool3d(x,kernel_size=(1,2,2),stride=(1,2,2)) #En Pytorch el orden de la tupla del kernel es (d,h,w)
        #Segunda Convolucion
        x = self.convolucion3d_2(x)
        x = F.relu(x)
        x = F.max_pool3d(x,kernel_size=(2,2,2),stride=(2,2,2))
        #Tercera Convolucion
        x = self.convolucion3d_3(x)
        x = F.relu(x)
        x = F.max_pool3d(x,kernel_size=(2,2,2),stride=(2,2,2))
        #Cuarta Convolucion
        x = self.convolucion3d_4(x)
        x = F.relu(x)
        x = F.max_pool3d(x,kernel_size=(2,2,2),stride=(2,2,2))
        #Quinta Convolucion
        x = self.convolucion3d_5(x)
        x = F.relu(x)
        x = F.max_pool3d(x,kernel_size=(2,2,2),stride=(2,2,2))
        #Conversion o ajuste de dimensiones para pasarla a la Full conected
        x = x.view(-1, self.numero_plano_caracteristicas(x)) #-1 para mantener correctamente la dimension de los batches
        #Primera Fullconected
        x = self.full_connected_1(x)
        x = F.relu(x)
        #Segunda Fullconected
        x = self.full_connected_2(x)
        x = F.relu(x)
        #Tercera Fullconected
        x = self.full_connected_3(x)
        x = F.log_softmax(x)
        return x
    
    def numero_plano_caracteristicas(self,x):
        tamaño = x.size()[1:] #Excluimos el tamaño N que corresponde a los batches
        num_caracteristicas = 1
        for dimension in tamaño:
            num_caracteristicas *= dimension
        return num_caracteristicas

## Definicion de Carga de Datos con el Modulo de PyTorch

In [ ]:
class FaceLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:].as_matrix()
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample